This notebook applies XPath expressions to Webnucleo XML data. The output is a new XML file with the XPath applied. Begin by running the whole notebook (under Kernel or Runtime, click Run all).  On Colab, you may need to sign into your Google account and to click on 'Run anyway' if an authorship warning comes up.

First install and import the necessary packages.

In [1]:
import sys
!{sys.executable} -m pip install --quiet wnutils 
!{sys.executable} -m pip install --quiet requests

import wnutils.xml as wx
import os, io, requests

Begin by reading in the XML. You may read the data over the web from an appropriate URL (the default for this notebook downloads the Webnucleo V2.2 Reaclib data from OSF) or from a local directory (as in the comment). If you are running the notebook on Google Colab, you can upload the data to the work directory by clicking on the directory tab to the left and then clicking the upload symbol. On Colab, you may also read the file from Google Drive (click the Drive icon and follow the instructions).

In [2]:
xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/kyhbs/download').content))
#xml= wx.Xml("example.xml")

Check the validity and type of the xml.

In [3]:
xml.validate()

allowed_types = {'nuclear_data', 'nuclear_network', 'libnucnet_input'}

if xml.get_type() not in allowed_types:
    print("Invalid input xml type for routine.")
    exit()

Choose the XPath expressions to apply.  The possible XPath expressions are on the nuclides, the reactions, and the zones, and the expressions will be applied in that order (for example, if the nuclide XPath expression excludes certain nuclides from the nuclear collection, it will also do so from the reactions and zones).  The default is to select only nuclei with atomic number less than or equal to 30, reactions, if present, that include neutrons as a reactant or electrons as a product, and, if present, the first 10 zones.  The XPath expression "" or "[.]" selects all nodes of the given type.

In [4]:
nuc_xpath = "[z <= 30]"
reac_xpath = "[reactant = 'n' or product = 'electron']"
zone_xpath = "[position() <= 10]"

Now create a new XML object and apply the XPath.

In [5]:
new_xml = wx.New_Xml(xml_type=xml.get_type())
nuclides = xml.get_nuclide_data(nuc_xpath)
new_xml.set_nuclide_data(nuclides)

if xml.get_type() != "nuclear_data":
    new_reactions = {}
    reactions = xml.get_reaction_data(reac_xpath)
    new_reactions = {}
    other_reaction_elements = ['gamma', 'electron', 'positron',
                               'neutrino_e', 'anti-neutrino_e',
                               'neutrino_mu', 'anti-neutrino_mu',
                               'neutrino_tau', 'anti-neutrino_tau']

    for reaction in reactions:
        b_valid = True
        for reactant in reactions[reaction].reactants:
            if reactant not in nuclides and \
               reactant not in other_reaction_elements:
                   b_valid = False
        for product in reactions[reaction].products:
            if product not in nuclides and \
               product not in other_reaction_elements:
                   b_valid = False
        if b_valid:
            new_reactions[reaction] = reactions[reaction]

    new_xml.set_reaction_data(new_reactions)

    if xml.get_type() != "nuclear_network":
        zones = xml.get_zone_data(zone_xpath)

        new_zones = {}

        for zone in zones:
            props = zones[zone]['properties']
            x = {}
            for sp in zones[zone]['mass fractions']:
                if sp[0] in nuclides:
                    x[sp] = zones[zone]['mass fractions'][sp]
            new_zones[zone] = {'properties': props, 'mass fractions': x}

        new_xml.set_zone_data(new_zones)


Choose an output file name.

In [6]:
my_file = "my_xpath.xml"

Write out the data to the file.  The new XML file exists in the working directory.  If you are running this notebook on Google Colab, you can download the file by clicking on the folder icon on the left to show the current files in your working directory.  To download the file, click on the file name and then choose download.

In [7]:
new_xml.write(my_file)